#### Load libraries

In [1]:
import numpy as np
import pandas as pd
from datetime import date
import matplotlib.pyplot as plt
import os
import math
from shapely.geometry import Point, Polygon
#%pip install rtree 
#%pip install pygeos 
import geopandas as gpd
import rtree
import pygeos

/Users/didi/Documents/Short Term Forecast/jupyterEnv/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/var/folders/qq/jd7pmms11gv5jnzmr9cwqgk40000gn/T/ipykernel_12492/2071315555.py:10: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyG

In [2]:
# function definition

# Define a function to convert the data to a Polygon
def convert_to_polygon(x):
    coords = [tuple(map(float, coord.split(','))) for coord in x.split(';')]
    return Polygon(coords)

def inside_polygon(df_X, polygons):
    # Transform the DataFrames to GeoPandas DataFrames
    df_X = gpd.GeoDataFrame(df_X, geometry='start_point')
    poly = gpd.GeoDataFrame(polygons, geometry='polygon')
    # Use sjoin to join the df and poly dataframes
    df_X = gpd.sjoin(df_X, poly, op='within', how='left')
    #drop unnecessary columns and rename the columns
    #df_X = df_X.drop(columns=['index_right', 'map_item','fence_id', 'name']).rename(columns={'fence_id_right': 'fence_id', 'name_right': 'name'})
    return df_X


#### Read csv file

In [3]:
# aggregated-results -> oxy
# extractions -> gdc

#provider_a = "oxy"
provider_b = "gdc"

# Adjust date
#date_name = "2022-06-06" ("yyyy-mm-dd") monday
date_name = "2023-04-17"

# Specify Week num
# week_num = "W24"
week_num = "202317" # Year Calendar Week for 2023

In [4]:
#input file name _b
#filename_b = "CI_inputs_"+date_name+"_"+provider_b+".csv"

#path_b = "inputs\\" + filename_b
path_b = "/Users/didi/Documents/Short Term Forecast/SSL CI Files/extractions-13-03-2023-to-19-03-2023.csv" 


df_raw_b = pd.read_csv(path_b)

# Read Polygon file
poly = pd.read_csv('/Users/didi/Documents/Short Term Forecast/SSL CI Files/ssl_polygons.csv' ,encoding= 'unicode_escape')
poly.map_item = poly.map_item.str.replace('_', ',')

#### Create new folder to store weekly data

In [5]:
#newpath = "outputs\\Weekly\\" + week_num + '\\'
newpath = "../SSL CI Files/Weekly/" + week_num + '/'

if not os.path.exists(newpath):
    os.makedirs(newpath)

#### Data preview

In [6]:
df_raw_b['start_date'] = date_name

df_raw_b['provider'] = provider_b

df_raw_b.head()

,row_id,date,time,weekday,country,city,start_lat,start_long,end_lat,end_long,...,price,base_fare,min_fare,fixed,per_km,per_km_2,per_minute,vbf,start_date,provider
0,r56900,13/03/2023,05:00,Friday,Mexico,Juarez(CHIH),31.613240,-106.34450,31.595680,-106.343772,...,48.22,10.00,30.18,0.0,5.00,0.9,2.45,5.09,2023-04-17,gdc
1,r70802,13/03/2023,05:00,Friday,Mexico,Monterrey,25.734530,-100.35223,25.742540,-100.359630,...,27.57,7.70,22.00,0.0,4.47,0.9,2.40,3.84,2023-04-17,gdc
2,r76186,13/03/2023,05:00,Friday,Mexico,Monterrey,25.800989,-100.22066,25.773890,-100.251210,...,94.62,7.70,22.00,0.0,4.47,0.9,2.40,8.81,2023-04-17,gdc
3,r1282,13/03/2023,05:00,Friday,Mexico,Chihuahua,28.609200,-106.05570,28.602950,-106.089465,...,53.02,5.42,25.20,0.0,4.05,0.8,1.73,5.66,2023-04-17,gdc
4,r16881,13/03/2023,05:00,Friday,Colombia,Bogota,4.583110,-74.10339,4.594421,-74.119039,...,9200.00,2150.00,6150.00,0.0,673.00,95.0,121.00,830.00,2023-04-17,gdc


In [7]:
df_raw_b.shape

(3477588, 22)

In [8]:
df_raw_b.describe()
print('Number of total Records per city:','\n',df_raw_b.city.value_counts(),'\n')
print('Number of record with 0 KM per country:','\n', df_raw_b.country[(df_raw_b.price == 0)].value_counts())

Number of total Records per city: 
 Mexico City             944110
Medellin                276464
Bogota                  276454
Monterrey               240939
Buenos Aires            187832
Juarez(CHIH)            182604
Cali                    138398
Guadalajara             132956
Culiacan                130086
San Jose                108492
Toluca                  103978
Hermosillo               91230
Chihuahua                91006
Queretaro                90322
Mexicali                 89978
Leon                     86568
Puebla                   73747
Lima                     59513
Tijuana                  58927
Santo Domingo            36508
Quito                    29228
Merida                   28762
Region Metropolitana     19486
Name: city, dtype: int64 

Number of record with 0 KM per country: 
 Mexico                59719
Colombia              17551
Argentina              4788
Costa Rica             2719
Peru                   1540
Dominican Republic      889
Ecuador       

##### List of cities

In [9]:
cities_b = list(df_raw_b["city"].unique())
print(cities_b)

['Juarez(CHIH)', 'Monterrey', 'Chihuahua', 'Bogota', 'Mexico City', 'Cali', 'Culiacan', 'San Jose', 'Leon', 'Medellin', 'Buenos Aires', 'Toluca', 'Lima', 'Guadalajara', 'Queretaro', 'Tijuana', 'Mexicali', 'Santo Domingo', 'Hermosillo', 'Puebla', 'Quito', 'Merida', 'Region Metropolitana']


##### List of products

In [10]:
products_b = list(df_raw_b["product"].unique())
print(products_b)

['UberX Promo', 'UberX', 'Moto', 'UberYa', 'Economy', 'UberYa Promo']


##### Standardizing date format

In [11]:
#df_raw_b['date'] = pd.to_datetime(df_raw_b['date']).dt.strftime('%Y-%m-%d')

df_raw_b['date'] = pd.to_datetime(df_raw_b['date'], format='%d/%m/%Y')

df_raw_b.head()

,row_id,date,time,weekday,country,city,start_lat,start_long,end_lat,end_long,...,price,base_fare,min_fare,fixed,per_km,per_km_2,per_minute,vbf,start_date,provider
0,r56900,2023-03-13,05:00,Friday,Mexico,Juarez(CHIH),31.613240,-106.34450,31.595680,-106.343772,...,48.22,10.00,30.18,0.0,5.00,0.9,2.45,5.09,2023-04-17,gdc
1,r70802,2023-03-13,05:00,Friday,Mexico,Monterrey,25.734530,-100.35223,25.742540,-100.359630,...,27.57,7.70,22.00,0.0,4.47,0.9,2.40,3.84,2023-04-17,gdc
2,r76186,2023-03-13,05:00,Friday,Mexico,Monterrey,25.800989,-100.22066,25.773890,-100.251210,...,94.62,7.70,22.00,0.0,4.47,0.9,2.40,8.81,2023-04-17,gdc
3,r1282,2023-03-13,05:00,Friday,Mexico,Chihuahua,28.609200,-106.05570,28.602950,-106.089465,...,53.02,5.42,25.20,0.0,4.05,0.8,1.73,5.66,2023-04-17,gdc
4,r16881,2023-03-13,05:00,Friday,Colombia,Bogota,4.583110,-74.10339,4.594421,-74.119039,...,9200.00,2150.00,6150.00,0.0,673.00,95.0,121.00,830.00,2023-04-17,gdc


### Merging data

In [12]:
#df_raw = pd.concat([df_raw_a, df_raw_b], axis=0)
df_raw = df_raw_b

df_raw

,row_id,date,time,weekday,country,city,start_lat,start_long,end_lat,end_long,...,price,base_fare,min_fare,fixed,per_km,per_km_2,per_minute,vbf,start_date,provider
0,r56900,2023-03-13,05:00,Friday,Mexico,Juarez(CHIH),31.613240,-106.344500,31.595680,-106.343772,...,48.22,10.00,30.18,0.0,5.00,0.90,2.45,5.09,2023-04-17,gdc
1,r70802,2023-03-13,05:00,Friday,Mexico,Monterrey,25.734530,-100.352230,25.742540,-100.359630,...,27.57,7.70,22.00,0.0,4.47,0.90,2.40,3.84,2023-04-17,gdc
2,r76186,2023-03-13,05:00,Friday,Mexico,Monterrey,25.800989,-100.220660,25.773890,-100.251210,...,94.62,7.70,22.00,0.0,4.47,0.90,2.40,8.81,2023-04-17,gdc
3,r1282,2023-03-13,05:00,Friday,Mexico,Chihuahua,28.609200,-106.055700,28.602950,-106.089465,...,53.02,5.42,25.20,0.0,4.05,0.80,1.73,5.66,2023-04-17,gdc
4,r16881,2023-03-13,05:00,Friday,Colombia,Bogota,4.583110,-74.103390,4.594421,-74.119039,...,9200.00,2150.00,6150.00,0.0,673.00,95.00,121.00,830.00,2023-04-17,gdc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3477583,r78945,2023-03-19,23:30,Tuesday,Mexico,Monterrey,25.899070,-100.161450,25.907200,-100.167800,...,39.96,7.70,22.00,0.0,4.47,0.90,2.40,5.57,2023-04-17,gdc
3477584,r24221,2023-03-19,23:30,Tuesday,Mexico,Mexico City,19.428639,-99.112606,19.413130,-98.924640,...,334.30,6.50,30.15,0.0,3.84,1.16,1.98,33.50,2023-04-17,gdc
3477585,r7207,2023-03-19,23:30,Tuesday,Costa Rica,San Jose,9.912506,-84.093033,9.911679,-84.065749,...,1900.00,460.00,863.00,0.0,215.00,14.80,42.00,84.70,2023-04-17,gdc
3477586,r97919,2023-03-19,23:30,Tuesday,Mexico,Toluca,19.295300,-99.654010,19.294562,-99.634013,...,40.26,6.50,23.40,0.0,3.67,0.57,1.45,2.15,2023-04-17,gdc


### Adding Variables

#### Standardizing time

In [13]:
# converting into hourly time slots

#df_raw_a['hour'] = pd.to_datetime(df_raw_a['time'], format='%H:%M').dt.hour
#df_raw_b['hour'] = pd.to_datetime(df_raw_b['time'], format='%H:%M').dt.hour

df_raw['hour'] = pd.to_datetime(df_raw['time'], format='%H:%M').dt.hour

#### Adding BF

In [14]:
# BF estimation: vbf / (price - vbf)

#df_raw_a["BF"] = df_raw_a["vbf"] / (df_raw_a["price"] - df_raw_a["vbf"])
#df_raw_b["BF"] = df_raw_b["vbf"] / (df_raw_b["price"] - df_raw_b["vbf"])

df_raw["BF"] = df_raw["vbf"] / (df_raw["price"] - df_raw["vbf"])

#### Distance Standarization

In [15]:
# Distance by KM standarization

#df_raw_a["KM"] = df_raw_a["distance_km"].apply(np.ceil)
#df_raw_b["KM"] = df_raw_b["distance_km"].apply(np.ceil)

df_raw = df_raw[df_raw.distance_km != 0.0]
df_raw = df_raw[df_raw.price != 0.0]
df_raw["KM"] = df_raw["distance_km"].apply(math.trunc)


conditions_km = [df_raw['KM'] > 30]
choices_km = ['>30']

df_raw['KM'] = np.select(conditions_km, choices_km, default = df_raw.KM)
df_raw.loc[:,['KM', 'distance_km']][df_raw['KM'] == "0"].head()
#t.query("KM==0")

,KM,distance_km
65,0,0.78
364,0,0.89
478,0,0.82
546,0,0.64
813,0,0.89


#### Type of DayStandarization

In [16]:
# Get BF ranges for each city
weekdays = list(df_raw["weekday"].unique())
print(weekdays)

['Friday', 'Thursday', 'Wednesday', 'Saturday', 'Sunday', 'Monday', 'Tuesday']


In [17]:
# Workdays vs weekends

#Create a list of our conditions
conditions = [
    (df_raw['weekday'] == 'Monday') | (df_raw['weekday'] == 'Tuesday') | (df_raw['weekday'] == 'Wednesday') | (df_raw['weekday'] == 'Thursday') | (df_raw['weekday'] == 'Friday')
    ,(df_raw['weekday'] == 'Saturday') | (df_raw['weekday'] == 'Sunday')
]

choices = ['workday', 'weekend']

df_raw['DayOfWeek'] = np.select(conditions, choices, default='workday')
df_raw.head()

,row_id,date,time,weekday,country,city,start_lat,start_long,end_lat,end_long,...,per_km,per_km_2,per_minute,vbf,start_date,provider,hour,BF,KM,DayOfWeek
0,r56900,2023-03-13,05:00,Friday,Mexico,Juarez(CHIH),31.613240,-106.34450,31.595680,-106.343772,...,5.00,0.9,2.45,5.09,2023-04-17,gdc,5,0.118015,2,workday
1,r70802,2023-03-13,05:00,Friday,Mexico,Monterrey,25.734530,-100.35223,25.742540,-100.359630,...,4.47,0.9,2.40,3.84,2023-04-17,gdc,5,0.161820,1,workday
2,r76186,2023-03-13,05:00,Friday,Mexico,Monterrey,25.800989,-100.22066,25.773890,-100.251210,...,4.47,0.9,2.40,8.81,2023-04-17,gdc,5,0.102669,6,workday
3,r1282,2023-03-13,05:00,Friday,Mexico,Chihuahua,28.609200,-106.05570,28.602950,-106.089465,...,4.05,0.8,1.73,5.66,2023-04-17,gdc,5,0.119510,6,workday
4,r16881,2023-03-13,05:00,Friday,Colombia,Bogota,4.583110,-74.10339,4.594421,-74.119039,...,673.00,95.0,121.00,830.00,2023-04-17,gdc,5,0.099164,2,workday


#### Filtering only UberX

In [18]:
df_X = df_raw[(df_raw["product"] == "UberX")|(df_raw["product"] == "UberYa")]

### Adding fence id

In [19]:
# Convert lng and lat to points
df_X['start_point'] = df_X.apply(lambda row: Point(row['start_long'], row['start_lat']), axis=1)

# Apply the function to the 'poly' column of the dataframe
poly['polygon'] = poly['map_item'].apply(convert_to_polygon)

# Create a new column in the dataframe with the fence_id
df_X = inside_polygon(df_X, poly)

# Print the dataframe
df_X.head()

/var/folders/qq/jd7pmms11gv5jnzmr9cwqgk40000gn/T/ipykernel_12492/2259399615.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_X['start_point'] = df_X.apply(lambda row: Point(row['start_long'], row['start_lat']), axis=1)
/Users/didi/Documents/Short Term Forecast/jupyterEnv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


,row_id,date,time,weekday,country,city,start_lat,start_long,end_lat,end_long,...,hour,BF,KM,DayOfWeek,start_point,index_right,fence_id,name,map_item,area
1,r70802,2023-03-13,05:00,Friday,Mexico,Monterrey,25.734530,-100.35223,25.742540,-100.359630,...,5,0.161820,1,workday,POINT (-100.35223 25.73453),NaN,NaN,NaN,NaN,NaN
2,r76186,2023-03-13,05:00,Friday,Mexico,Monterrey,25.800989,-100.22066,25.773890,-100.251210,...,5,0.102669,6,workday,POINT (-100.22066 25.80099),NaN,NaN,NaN,NaN,NaN
4,r16881,2023-03-13,05:00,Friday,Colombia,Bogota,4.583110,-74.10339,4.594421,-74.119039,...,5,0.099164,2,workday,POINT (-74.10339 4.58311),NaN,NaN,NaN,NaN,NaN
10,r4486,2023-03-13,05:00,Friday,Colombia,Bogota,4.744330,-74.09486,4.698472,-74.062657,...,5,0.111111,8,workday,POINT (-74.09486 4.74433),NaN,NaN,NaN,NaN,NaN
13,r4715,2023-03-13,05:00,Friday,Costa Rica,San Jose,9.898010,-84.06438,9.895010,-84.070687,...,5,0.024334,1,workday,POINT (-84.06438 9.89801),NaN,NaN,NaN,NaN,NaN


In [20]:
df_X.columns

Index(['row_id', 'date', 'time', 'weekday', 'country', 'city', 'start_lat',
       'start_long', 'end_lat', 'end_long', 'distance_km', 'product', 'price',
       'base_fare', 'min_fare', 'fixed', 'per_km', 'per_km_2', 'per_minute',
       'vbf', 'start_date', 'provider', 'hour', 'BF', 'KM', 'DayOfWeek',
       'start_point', 'index_right', 'fence_id', 'name', 'map_item', 'area'],
      dtype='object')

In [21]:
df_X.head(20)

,row_id,date,time,weekday,country,city,start_lat,start_long,end_lat,end_long,...,hour,BF,KM,DayOfWeek,start_point,index_right,fence_id,name,map_item,area
1,r70802,2023-03-13,05:00,Friday,Mexico,Monterrey,25.734530,-100.35223,25.742540,-100.359630,...,5,0.161820,1,workday,POINT (-100.35223 25.73453),NaN,NaN,NaN,NaN,NaN
2,r76186,2023-03-13,05:00,Friday,Mexico,Monterrey,25.800989,-100.22066,25.773890,-100.251210,...,5,0.102669,6,workday,POINT (-100.22066 25.80099),NaN,NaN,NaN,NaN,NaN
4,r16881,2023-03-13,05:00,Friday,Colombia,Bogota,4.583110,-74.10339,4.594421,-74.119039,...,5,0.099164,2,workday,POINT (-74.10339 4.58311),NaN,NaN,NaN,NaN,NaN
10,r4486,2023-03-13,05:00,Friday,Colombia,Bogota,4.744330,-74.09486,4.698472,-74.062657,...,5,0.111111,8,workday,POINT (-74.09486 4.74433),NaN,NaN,NaN,NaN,NaN
13,r4715,2023-03-13,05:00,Friday,Costa Rica,San Jose,9.898010,-84.06438,9.895010,-84.070687,...,5,0.024334,1,workday,POINT (-84.06438 9.89801),NaN,NaN,NaN,NaN,NaN
16,r76060,2023-03-13,05:00,Friday,Mexico,Monterrey,25.630500,-100.28128,25.654670,-100.362710,...,5,0.070793,10,workday,POINT (-100.28128 25.63050),NaN,NaN,NaN,NaN,NaN
19,r9215,2023-03-13,05:00,Friday,Argentina,Buenos Aires,-34.619100,-58.37150,-34.578246,-58.425015,...,5,0.072792,10,workday,POINT (-58.37150 -34.61910),NaN,NaN,NaN,NaN,NaN
20,r76923,2023-03-13,05:00,Friday,Mexico,Monterrey,25.679650,-100.25326,25.706413,-100.244437,...,5,0.138808,4,workday,POINT (-100.25326 25.67965),NaN,NaN,NaN,NaN,NaN
21,r97922,2023-03-13,05:00,Friday,Mexico,Toluca,19.288600,-99.62554,19.272340,-99.616120,...,5,0.085354,4,workday,POINT (-99.62554 19.28860),NaN,NaN,NaN,NaN,NaN
23,r12170,2023-03-13,05:00,Friday,Argentina,Buenos Aires,-34.614700,-58.55070,-34.629346,-58.534007,...,5,0.077282,2,workday,POINT (-58.55070 -34.61470),NaN,NaN,NaN,NaN,NaN


In [22]:
tij = df_X[(df_X.city == 'Mexico City')].price
print('Min: ',tij.min())
print('Max: ',tij.max())
print('Count: ',tij.count())
print('Average: ',tij.mean())
tij.value_counts()

Min:  29.9
Max:  2040.45
Count:  460399
Average:  88.1904929854322


49.95     1726
49.99     1693
49.90     1679
49.93     1676
49.98     1670
          ... 
185.41       1
478.00       1
345.34       1
407.75       1
265.80       1
Name: price, Length: 25545, dtype: int64

## Fare analysis summary

In [23]:
#list of variables to get

variables = ['price', 'base_fare', 'min_fare',
       'per_km', 'per_minute', 'vbf', 'BF']

#list of variables to group by. You can add as many variables as you want
grouping = ['city', 'provider']

#list of calculations you want to see. You can include more than once
calculations = [np.mean, np.min, np.max]


df_fares = pd.pivot_table(df_X, values = variables, index = grouping, aggfunc=calculations)

df_fares = df_fares.reset_index()

#df_fares.columns = df_fares.columns.droplevel(1)

df_fares['start_date'] =  date_name

df_fares['DayOfWeek'] = 'All'

df_fares.rename(columns = {'price':'ASP'}, inplace = True)

df_fares

city provider      mean                            \
                                         BF    base_fare     min_fare   
0                 Bogota      gdc  0.105090  2151.355787  6154.252151   
1           Buenos Aires      gdc  0.087488   202.691009   468.470659   
2              Chihuahua      gdc  0.094525     8.000705    23.781926   
3               Culiacan      gdc  0.067885     8.007264    25.850000   
4            Guadalajara      gdc  0.078760     9.801562    35.005494   
5             Hermosillo      gdc  0.103367     9.503972    22.117084   
6           Juarez(CHIH)      gdc  0.106395    10.000000    30.180000   
7                   Leon      gdc  0.067599     9.250679    33.250000   
8                   Lima      gdc  0.030363     3.886807     7.202446   
9               Medellin      gdc  0.113478  2483.212994  5400.000000   
10                Merida      gdc  0.073466     9.780000    24.150000   
11              Mexicali      gdc  0.013909    12.000000    37.430000   
12           Mexico City      gdc  0.124807     6.872664    30.175851   
13             Monterrey      gdc  0.105386     7.717269    22.385595   
14                Puebla      gdc  0.092661     7.502405    26.230000   
15             Queretaro      gdc  0.086107     8.500000    30.310000   
16                 Quito      gdc  0.051035     0.620000     2.000000   
17  Region Metropolitana      gdc  0.117786   549.155401  1448.542061   
18              San Jose      gdc  0.040063   460.000000   863.000000   
19         Santo Domingo      gdc  0.061092    39.470000    97.000000   
20               Tijuana      gdc  0.001726     9.505214    33.573354   
21                Toluca      gdc  0.075661     8.454772    29.900000   

                                                           amin  ...          \
        per_km  per_minute           ASP          vbf        BF  ...     vbf   
0   673.577856  121.087224  13035.545106  1147.114835  0.026416  ...  500.00   
1    44.149704   35.342109    944.144792    73.436659  0.014676  ...   23.61   
2     4.100361    1.780159     61.677741     5.162231  0.018487  ...    1.65   
3     4.650000    2.050000     59.931358     3.683011  0.014461  ...    1.23   
4     4.854489    2.101814     92.070997     6.332460  0.012659  ...    2.00   
5     4.101714    2.321004     68.866342     6.154797  0.026320  ...    2.10   
6     5.000000    2.450000     75.643803     7.060123  0.021635  ...    2.50   
7     4.609942    1.999854     72.857029     4.499792  0.013867  ...    1.65   
8     0.919962    0.190019     14.690969     0.413229  0.007220  ...    0.19   
9   786.000000  141.000000  12310.401654  1203.715602  0.027318  ...  440.00   
10    4.190000    1.710000     74.186693     4.947276  0.017409  ...    1.50   
11    5.400000    2.830000     87.256969     1.138471  0.000000  ...    0.00   
12    4.152508    2.251529     88.190493     8.817355  0.014051  ...    2.62   
13    4.484573    2.400237     94.703794     8.348985  0.013986  ...    2.38   
14    3.950000    1.920000     74.121200     5.984589  0.022537  ...    2.10   
15    4.500000    2.130000     85.910241     6.514126  0.017304  ...    2.00   
16    0.230000    0.060000      3.531542     0.169930  0.010101  ...    0.05   
17  302.478469  107.167035   4233.920246   449.406875  0.035508  ...   96.00   
18  215.000000   42.000000   2600.190814    95.724180  0.010563  ...   26.33   
19   11.000000    5.000000    195.828756    10.944543  0.013691  ...    4.76   
20    4.850000    2.870000     88.901346     0.082799  0.000000  ...    0.00   
21    4.150000    1.530341     70.864902     4.868009  0.016007  ...    1.62   

        amax                                                            \
          BF base_fare min_fare  per_km per_minute        ASP      vbf   
0   0.266667   2360.00  6930.00  779.00     137.00  101900.00  4440.00   
1   0.322373    435.05   928.40   62.80      71.34   10470.00   741.33   
2   0.251252      8.80    60.00    4.51       1.96

#### Save output

In [24]:
# filename = "fares_WholeWeek_" + date_name + ".csv"

# #Save to fares folder
# #df_fares.to_csv("outputs\\fares\\" + filename, encoding="utf8", index = False)
# df_fares.to_csv("../outputs/fares/" + filename, encoding="utf8", index = False)

#### Fare components by type of day

In [25]:
#list of variables to get

variables = ['price', 'base_fare', 'min_fare',
       'per_km', 'per_minute', 'vbf', 'BF']

#list of variables to group by. You can add as many variables as you want
grouping = ['city', 'provider', 'DayOfWeek']

#list of calculations you want to see. You can include more than once
calculations = [np.mean, np.min, np.max]


df_fares_wd = pd.pivot_table(df_X, values = variables, index = grouping, aggfunc=calculations)

df_fares_wd = df_fares_wd.reset_index()

#df_fares.columns = df_fares.columns.droplevel(1)

df_fares_wd['start_date'] =  date_name

df_fares_wd.rename(columns = {'price':'ASP'}, inplace = True)

DayOfWeek = df_fares_wd['DayOfWeek']

df_fares_wd = df_fares_wd.drop(columns=['DayOfWeek'], level = 0)

df_fares_wd.insert(loc=24, column='DayOfWeek', value = DayOfWeek)

df_fares_wd.head()

city provider      mean                                        \
                                BF    base_fare     min_fare      per_km   
0        Bogota      gdc  0.105212  2151.353859  6154.284475  673.582249   
1        Bogota      gdc  0.105042  2151.356555  6154.239277  673.576107   
2  Buenos Aires      gdc  0.101165   214.845305   496.515912   16.275323   
3  Buenos Aires      gdc  0.082074   197.879633   457.368721   55.184000   
4     Chihuahua      gdc  0.094386     8.000701    23.781745    4.100359   

                                              amin  ...              amax  \
   per_minute           ASP          vbf        BF  ...     vbf        BF   
0  121.087887  13044.633998  1149.527031  0.026416  ...  500.00  0.238571   
1  121.086960  13031.925039  1146.154068  0.027224  ...  500.00  0.266667   
2   67.990640    953.179706    84.927904  0.017334  ...   30.69  0.244286   
3   22.417927    940.568249    68.887758  0.014676  ...   23.61  0.322373   
4    1.780158     61.782005     5.158346  0.020563  ...    1.65  0.206013   

                                                             start_date  \
  base_fare min_fare  per_km per_minute        ASP      vbf               
0   2360.00   6930.0  779.00     137.00  101900.00  4440.00  2023-04-17   
1   2360.00   6930.0  779.00     137.00   95900.00  4440.00  2023-04-17   
2    435.05    928.4   23.81      71.34   10470.00   741.33  2023-04-17   
3    435.05    928.4   62.80      71.34    9050.00   691.12  2023-04-17   
4      8.80     60.0    4.51       1.96     335.77    26.01  2023-04-17   

  DayOfWeek  
             
0   weekend  
1   workday  
2   weekend  
3   workday  
4   weekend  

[5 rows x 25 columns]

In [26]:
# filename = "fares_typeofday_" + date_name + ".csv"

# # Save fares to a new file
# #df_fares_wd.to_csv("outputs\\fares\\" + filename, encoding="utf8", index = False)

# df_fares_wd.to_csv("../outputs/fares/" + filename, encoding="utf8", index = False)

In [27]:
## concat all fares

df_fares_all = pd.concat([df_fares, df_fares_wd], axis=0)

df_fares_all.head()


city provider      mean                                        \
                                BF    base_fare     min_fare      per_km   
0        Bogota      gdc  0.105090  2151.355787  6154.252151  673.577856   
1  Buenos Aires      gdc  0.087488   202.691009   468.470659   44.149704   
2     Chihuahua      gdc  0.094525     8.000705    23.781926    4.100361   
3      Culiacan      gdc  0.067885     8.007264    25.850000    4.650000   
4   Guadalajara      gdc  0.078760     9.801562    35.005494    4.854489   

                                              amin  ...              amax  \
   per_minute           ASP          vbf        BF  ...     vbf        BF   
0  121.087224  13035.545106  1147.114835  0.026416  ...  500.00  0.266667   
1   35.342109    944.144792    73.436659  0.014676  ...   23.61  0.322373   
2    1.780159     61.677741     5.162231  0.018487  ...    1.65  0.251252   
3    2.050000     59.931358     3.683011  0.014461  ...    1.23  0.530536   
4    2.101814     92.070997     6.332460  0.012659  ...    2.00  0.236800   

                                                             start_date  \
  base_fare min_fare  per_km per_minute        ASP      vbf               
0   2360.00  6930.00  779.00     137.00  101900.00  4440.00  2023-04-17   
1    435.05   928.40   62.80      71.34   10470.00   741.33  2023-04-17   
2      8.80    60.00    4.51       1.96     335.77    26.02  2023-04-17   
3     11.00    25.85    4.65       2.05     360.49    21.02  2023-04-17   
4     10.00   135.00    5.84       2.50     875.25    44.37  2023-04-17   

  DayOfWeek  
             
0       All  
1       All  
2       All  
3       All  
4       All  

[5 rows x 25 columns]

In [28]:
# Save to new csv file
filename = "fares_all_" + date_name + ".csv"

#df_fares_all.to_csv("outputs\\fares\\" + filename, encoding="utf8", index = False)
#df_fares_all.to_csv("../outputs/fares/" + filename, encoding="utf8", index = False)


#Save to Weekly data folder

df_fares_all.to_csv(newpath + filename, encoding="utf8", index = False)

# Append fares to existing csv file with historical fares

# filename = "fares_hist.csv"

#df_fares_all.to_csv("outputs\\fares\\" + filename,                     # Add new data vertically
# df_fares_all.to_csv("../outputs/fares/" + filename,                     # Add new data vertically

#                     mode = 'a',
#                     header = False,
#                     encoding="utf8", 
#                     index = False)

## Fare analysis by distance

In [29]:
# ASP estimation by distance

#list of variables
#variables = ['price', 'theoric_price']
variables = ['price','vbf','BF']

#list of variables to group by
grouping = ['city', 'provider','KM']

#list of calculations
calculations = [np.mean, np.min, np.max, 'count']


# DF

#merged dataframe by both providers

df_KM_metrics = pd.pivot_table(df_X, values = variables, index = grouping, aggfunc = calculations)

df_KM_metrics = df_KM_metrics.reset_index()

df_KM_metrics['date'] = date_name

df_KM_metrics.rename(columns = {'price':'ASP'}, inplace = True)

df_KM_metrics

city provider   KM      mean                                 amin  \
                                 BF           ASP          vbf        BF   
0    Bogota      gdc    1  0.090812   8707.380952   672.779349  0.029293   
1    Bogota      gdc   10  0.108606  16263.449048  1474.138858  0.028774   
2    Bogota      gdc   11  0.107031  17493.531426  1563.105894  0.028668   
3    Bogota      gdc   12  0.109399  17939.242853  1641.344321  0.030943   
4    Bogota      gdc   13  0.108487  19142.295858  1736.390533  0.031963   
..      ...      ...  ...       ...           ...          ...       ...   
651  Toluca      gdc    6  0.078467     72.703866     5.123522  0.019019   
652  Toluca      gdc    7  0.081024     77.058291     5.611978  0.020801   
653  Toluca      gdc    8  0.080228     86.455045     6.187000  0.022409   
654  Toluca      gdc    9  0.082139     91.514581     6.776623  0.027549   
655  Toluca      gdc  >30  0.060010    651.158466    31.945511  0.035876   

                          amax                     count                \
         ASP     vbf        BF       ASP      vbf     BF    ASP    vbf   
0    6150.00  500.00  0.171846  63500.00  3390.00  12726  12726  12726   
1    6280.00  500.00  0.244928  49760.00  2180.00   4465   4465   4465   
2    6190.00  740.00  0.214529  54900.00  2080.00   4089   4089   4089   
3    6370.00  530.00  0.238035  56640.00  2260.00   3883   3883   3883   
4    6600.00  560.00  0.219849  59280.00  2530.00   3380   3380   3380   
..       ...     ...       ...       ...      ...    ...    ...    ...   
651    33.84    1.66  0.169928    189.91     9.74   3634   3634   3634   
652    30.92    1.62  0.160765    179.99     8.37   2856   2856   2856   
653    30.92    1.63  0.192498    269.91    11.29   2357   2357   2357   
654    35.00    1.62  0.134179    209.99     9.53   1919   1919   1919   
655   139.84   12.41  0.154487   1690.62    67.09    176    176    176   

           date  
                 
0    2023-04-17  
1    2023-04-17  
2    2023-04-17  
3    2023-04-17  
4    2023-04-17  
..          ...  
651  2023-04-17  
652  2023-04-17  
653  2023-04-17  
654  2023-04-17  
655  2023-04-17  

[656 rows x 16 columns]

In [30]:
# Save fares to new csv file 
filename = "KM_metrics_" + date_name + ".csv"

#df_KM_metrics.to_csv("outputs\\fares\\fares_KM\\" + filename, encoding="utf8",  index=False)
#df_KM_metrics.to_csv("../outputs/fares/fares_KM/" + filename, encoding="utf8",  index=False)



#Save to Weekly data folder

df_KM_metrics.to_csv(newpath + filename, encoding="utf8", index = False)


# Append fares to existing csv file with historical fares

#filename = "KM_metrics_hist.csv"

#df_KM_metrics.to_csv("outputs\\fares\\fares_KM\\" + filename,                     # Add new data vertically
# df_KM_metrics.to_csv("../outputs/fares/fares_KM/" + filename,                     # Add new data vertically

#                     mode = 'a',
#                     header = False,
#                     encoding="utf8", 
#                     index = False)

## Fare Analysis by day of the week

In [31]:
#list of variables to get
#variables = ['price', 'vbf', 'BF']
variables = ['price', 'base_fare', 'min_fare',
       'per_km', 'per_minute', 'vbf', 'BF']

#list of variables to group by. You can add as many variables as you want
grouping = ['city', 'provider','weekday']

#list of calculations you want to see. You can include more than once
calculations = [np.mean, np.min, np.max]

df_weekdays = pd.pivot_table(df_X, values = variables, index = grouping, aggfunc=calculations)

df_weekdays = df_weekdays.reset_index()

df_weekdays['date'] = date_name

df_weekdays.rename(columns = {'price':'ASP'}, inplace = True)

df_weekdays

city provider    weekday      mean                            \
                                       BF    base_fare     min_fare   
0    Bogota      gdc     Friday  0.105086  2151.361446  6154.206540   
1    Bogota      gdc     Monday  0.104995  2151.369776  6154.277303   
2    Bogota      gdc   Saturday  0.105103  2151.345322  6154.251756   
3    Bogota      gdc     Sunday  0.105327  2151.362906  6154.319143   
4    Bogota      gdc   Thursday  0.105265  2151.311704  6154.162523   
..      ...      ...        ...       ...          ...          ...   
149  Toluca      gdc   Saturday  0.075630     9.049113    29.900000   
150  Toluca      gdc     Sunday  0.075505     9.048755    29.900000   
151  Toluca      gdc   Thursday  0.075555     8.219613    29.900000   
152  Toluca      gdc    Tuesday  0.075741     8.219269    29.900000   
153  Toluca      gdc  Wednesday  0.075592     8.219274    29.900000   

                                                        ...     amin          \
         per_km  per_minute           ASP          vbf  ...      ASP     vbf   
0    673.571658  121.086288  13008.618474  1146.638554  ...  6150.00  500.00   
1    673.581275  121.087740  13031.286077  1145.708843  ...  6150.00  500.00   
2    673.577803  121.087216  13065.253723  1150.485530  ...  6150.00  500.00   
3    673.586960  121.088598  13022.786544  1148.511462  ...  6150.00  500.00   
4    673.565676  121.085385  12989.445725  1144.460279  ...  6150.00  500.00   
..          ...         ...           ...          ...  ...      ...     ...   
149    4.150000    1.530293     70.721620     4.859132  ...    29.91    1.62   
150    4.150000    1.530411     71.178527     4.880670  ...    29.93    1.62   
151    4.150000    1.530206     70.799137     4.857222  ...    29.94    1.62   
152    4.150000    1.530390     70.828017     4.869174  ...    29.95    1.62   
153    4.150000    1.530387     71.086540     4.871522  ...    29.98    1.62   

         amax                                                            \
           BF base_fare min_fare  per_km per_minute        ASP      vbf   
0    0.229249   2360.00   6930.0  779.00     137.00   91400.00  4250.00   
1    0.244928   2360.00   6930.0  779.00     137.00   95900.00  4260.00   
2    0.238571   2360.00   6930.0  779.00     137.00   88800.00  4440.00   
3    0.227737   2360.00   6930.0  779.00     137.00  101900.00  4260.00   
4    0.248988   2360.00   6930.0  779.00     137.00   91300.00  4440.00   
..        ...       ...      ...     ...        ...        ...      ...   
149  0.185926      9.05     29.9    4.15       2.25    1409.93    66.61   
150  0.162187      9.05     29.9    4.15       2.25    1690.62    67.09   
151  0.164338      8.22     29.9    4.15       2.25    1367.37    66.81   
152  0.171607      8.22     29.9    4.15       2.25    1399.97    62.25   
153  0.192498      8.22     29.9    4.15       2.25    1444.94    66.88   

           date  
                 
0    2023-04-17  
1    2023-04-17  
2    2023-04-17  
3    2023-04-17  
4    2023-04-17  
..          ...  
149  2023-04-17  
150  2023-04-17  
151  2023-04-17  
152  2023-04-17  
153  2023-04-17  

[154 rows x 25 columns]

In [32]:
# Save fares to new csv file df_weekdays

filename = "fares_weekday_" + date_name + ".csv"

#df_weekdays.to_csv("outputs\\fares\\fares_weekday\\" + filename, encoding="utf8", index = False)
#df_weekdays.to_csv("../outputs/fares/fares_weekday/" + filename, encoding="utf8", index = False)



#Save to Weekly data folder

df_weekdays.to_csv(newpath + filename, encoding="utf8", index = False)


# Append fares to existing csv file with historical fares
#filename = "fares_weekday_hist.csv"

#df_weekdays.to_csv("outputs\\fares\\fares_weekday\\" + filename,                     # Add new data vertically
# df_weekdays.to_csv("../outputs/fares/fares_weekday/" + filename,                     # Add new data vertically
#                     mode = 'a',
#                     header = False,
#                     encoding="utf8", 
#                     index = False)

# Hourly View

In [33]:
#list of variables to get
#variables = ['price', 'vbf', 'BF']
variables = ['price', 'base_fare', 'min_fare',
       'per_km', 'per_minute', 'vbf', 'BF']

#list of variables to group by. You can add as many variables as you want
grouping = ['city', 'provider','weekday', 'hour']

#list of calculations you want to see. You can include more than once
calculations = [np.mean, 'count']

df_hour = pd.pivot_table(df_X, values = variables, index = grouping, aggfunc=calculations)

df_hour = df_hour.reset_index()

df_hour['date'] = date_name

df_hour.rename(columns = {'price':'ASP'}, inplace = True)

df_hour

city provider    weekday hour      mean                            \
                                             BF    base_fare     min_fare   
0     Bogota      gdc     Friday    5  0.105796  2151.235294  6154.588235   
1     Bogota      gdc     Friday    6  0.104146  2152.076271  6157.711864   
2     Bogota      gdc     Friday    7  0.103976  2152.082153  6157.733711   
3     Bogota      gdc     Friday    8  0.106456  2151.200000  6154.457143   
4     Bogota      gdc     Friday    9  0.105958  2151.609195  6155.977011   
...      ...      ...        ...  ...       ...          ...          ...   
2921  Toluca      gdc  Wednesday   19  0.074921     8.220000    29.900000   
2922  Toluca      gdc  Wednesday   20  0.075115     8.220000    29.900000   
2923  Toluca      gdc  Wednesday   21  0.075148     8.220000    29.900000   
2924  Toluca      gdc  Wednesday   22  0.074246     8.220000    29.900000   
2925  Toluca      gdc  Wednesday   23  0.074477     8.220000    29.900000   

                                                        count            \
          per_km  per_minute           ASP          vbf    BF base_fare   
0     673.623529  121.094118  12379.058824  1103.411765   340       340   
1     674.048023  121.158192  12885.211864  1138.644068   708       708   
2     674.050992  121.158640  13067.096317  1138.895184   706       706   
3     673.605714  121.091429  13154.685714  1177.200000   525       525   
4     673.812261  121.122605  13178.275862  1181.302682   522       522   
...          ...         ...           ...          ...   ...       ...   
2921    4.150000    1.530000     70.237212     4.800965   373       373   
2922    4.150000    1.530000     73.369290     4.932534   521       521   
2923    4.150000    1.530000     68.015010     4.656411   521       521   
2924    4.150000    1.530000     73.386756     4.972279   373       373   
2925    4.150000    1.530000     69.696022     4.756210   372       372   

                                                 date  
     min_fare per_km per_minute  ASP  vbf              
0         340    340        340  340  340  2023-04-17  
1         708    708        708  708  708  2023-04-17  
2         706    706        706  706  706  2023-04-17  
3         525    525        525  525  525  2023-04-17  
4         522    522        522  522  522  2023-04-17  
...       ...    ...        ...  ...  ...         ...  
2921      373    373        373  373  373  2023-04-17  
2922      521    521        521  521  521  2023-04-17  
2923      521    521        521  521  521  2023-04-17  
2924      373    373        373  373  373  2023-04-17  
2925      372    372        372  372  372  2023-04-17  

[2926 rows x 19 columns]

In [34]:
# Save fares to new csv file df_weekdays

filename = "fares_hour_" + date_name + ".csv"

#df_weekdays.to_csv("outputs\\fares\\fares_weekday\\" + filename, encoding="utf8", index = False)
#df_hour.to_csv("../outputs/fares/fares_hourly/" + filename, encoding="utf8", index = False)


#Save to Weekly data folder
df_hour.to_csv(newpath + filename, encoding="utf8", index = False)


# Append fares to existing csv file with historical fares
#filename = "fares_hour_hist.csv"

#df_weekdays.to_csv("outputs\\fares\\fares_weekday\\" + filename,                     # Add new data vertically
# df_hour.to_csv("../outputs/fares/fares_hourly/" + filename,                     # Add new data vertically
#                     mode = 'a',
#                     header = False,
#                     encoding="utf8", 
#                     index = False)

# Geofence Data

In [35]:
df_X.columns

Index(['row_id', 'date', 'time', 'weekday', 'country', 'city', 'start_lat',
       'start_long', 'end_lat', 'end_long', 'distance_km', 'product', 'price',
       'base_fare', 'min_fare', 'fixed', 'per_km', 'per_km_2', 'per_minute',
       'vbf', 'start_date', 'provider', 'hour', 'BF', 'KM', 'DayOfWeek',
       'start_point', 'index_right', 'fence_id', 'name', 'map_item', 'area'],
      dtype='object')

In [36]:
#list of variables to get
#variables = ['price', 'vbf', 'BF']
variables = ['price', 'base_fare', 'min_fare',
       'per_km', 'per_minute', 'vbf', 'BF']

df_geo = df_X[df_X.fence_id != 0]

#list of variables to group by. You can add as many variables as you want
grouping = ['city', 'provider','weekday', 'fence_id', 'name']

#list of calculations you want to see. You can include more than once
calculations = [np.mean, 'count']

df_geo = pd.pivot_table(df_geo, values = variables, index = grouping, aggfunc=calculations)

df_geo = df_geo.reset_index()

df_geo['date'] = date_name

df_geo.rename(columns = {'price':'ASP'}, inplace = True)

df_geo

city provider    weekday   fence_id                        name  \
                                                                              
0         Bogota      gdc     Friday  3018041.0  Geofence NE and W Ajustado   
1         Bogota      gdc     Friday  3018472.0            Geopricing South   
2         Bogota      gdc     Monday  3018041.0  Geofence NE and W Ajustado   
3         Bogota      gdc     Monday  3018472.0            Geopricing South   
4         Bogota      gdc   Saturday  3018041.0  Geofence NE and W Ajustado   
..           ...      ...        ...        ...                         ...   
100  Mexico City      gdc  Wednesday   107199.0        Central_CDMX_Pricing   
101  Mexico City      gdc  Wednesday   107203.0          North_CDMX_Pricing   
102  Mexico City      gdc  Wednesday   107207.0          South_CDMX_Pricing   
103  Mexico City      gdc  Wednesday   107211.0      Southeast_CDMX_Pricing   
104  Mexico City      gdc  Wednesday   107215.0           West_CDMX_Pricing   

         mean                                                                  \
           BF    base_fare     min_fare      per_km  per_minute           ASP   
0    0.099962  2150.666667  6150.000000  673.000000  121.000000  14448.238095   
1    0.110308  2150.000000  6150.000000  673.000000  121.000000  12557.217391   
2    0.099886  2150.834658  6150.000000  673.000000  121.000000  14526.899841   
3    0.113840  2150.000000  6150.000000  673.000000  121.000000  12217.892377   
4    0.099962  2150.641221  6150.000000  673.000000  121.000000  14546.038168   
..        ...          ...          ...         ...         ...           ...   
100  0.107199     6.879477    30.918736    4.159261    2.255647    102.453215   
101  0.131286     6.870000    29.900000    4.150000    2.250000     79.241821   
102  0.115828     6.870000    29.900000    4.150000    2.250000    101.438730   
103  0.134868     6.870000    29.900000    4.150000    2.250000     78.865916   
104  0.111768     6.870000    29.900000    4.150000    2.250000    105.032132   

                  count                                                     \
             vbf     BF base_fare min_fare per_km per_minute    ASP    vbf   
0    1222.261905   1260      1260     1260   1260       1260   1260   1260   
1    1185.130435    230       230      230    230        230    230    230   
2    1218.783784   1258      1258     1258   1258       1258   1258   1258   
3    1199.058296    223       223      223    223        223    223    223   
4    1226.328244   1310      1310     1310   1310       1310   1310   1310   
..           ...    ...       ...      ...    ...        ...    ...    ...   
100     8.809988  18328     18328    18328  18328      18328  18328  18328   
101     8.467162  16472     16472    16472  16472      16472  16472  16472   
102     9.557985   6614      6614     6614   6614       6614   6614   6614   
103     8.729156  13549     13549    13549  13549      13549  13549  13549   
104     9.596349   2824      2824     2824   2824       2824   2824   2824   

           date  
                 
0    2023-04-17  
1    2023-04-17  
2    2023-04-17  
3    2023-04-17  
4    2023-04-17  
..          ...  
100  2023-04-17  
101  2023-04-17  
102  2023-04-17  
103  2023-04-17  
104  2023-04-17  

[105 rows x 20 columns]

In [37]:
# Save fares to new csv file df_weekdays

filename = "fares_geo_" + date_name + ".csv"

#df_weekdays.to_csv("outputs\\fares\\fares_weekday\\" + filename, encoding="utf8", index = False)
#df_geo.to_csv("../outputs/fares/fares_geo/" + filename, encoding="utf8", index = False)


#Save to Weekly data folder
df_geo.to_csv(newpath + filename, encoding="utf8", index = False)


# Append fares to existing csv file with historical fares
#filename = "fares_geo_hist.csv"

#df_weekdays.to_csv("outputs\\fares\\fares_weekday\\" + filename,                     # Add new data vertically
# df_geo.to_csv("../outputs/fares/fares_geo/" + filename,                     # Add new data vertically
#                     mode = 'a',
#                     header = False,
#                     encoding="utf8", 
#                     index = False)

In [38]:
df_geo.shape

(105, 20)

In [39]:
#list of variables to get
#variables = ['price', 'vbf', 'BF']
variables = ['price', 'base_fare', 'min_fare',
       'per_km', 'per_minute', 'vbf', 'BF']

df_geo_full = df_X[df_X.fence_id != 0]

#list of variables to group by. You can add as many variables as you want
grouping = ['city', 'provider','weekday', 'fence_id', 'name', 'hour','KM' ]

#list of calculations you want to see. You can include more than once
calculations = [np.mean, 'count']

df_geo_full = pd.pivot_table(df_geo_full, values = variables, index = grouping, aggfunc=calculations)

df_geo_full = df_geo_full.reset_index()

df_geo_full['date'] = date_name

df_geo_full.rename(columns = {'price':'ASP'}, inplace = True)

df_geo_full

city provider    weekday   fence_id                        name  \
                                                                                
0           Bogota      gdc     Friday  3018041.0  Geofence NE and W Ajustado   
1           Bogota      gdc     Friday  3018041.0  Geofence NE and W Ajustado   
2           Bogota      gdc     Friday  3018041.0  Geofence NE and W Ajustado   
3           Bogota      gdc     Friday  3018041.0  Geofence NE and W Ajustado   
4           Bogota      gdc     Friday  3018041.0  Geofence NE and W Ajustado   
...            ...      ...        ...        ...                         ...   
39865  Mexico City      gdc  Wednesday   107215.0           West_CDMX_Pricing   
39866  Mexico City      gdc  Wednesday   107215.0           West_CDMX_Pricing   
39867  Mexico City      gdc  Wednesday   107215.0           West_CDMX_Pricing   
39868  Mexico City      gdc  Wednesday   107215.0           West_CDMX_Pricing   
39869  Mexico City      gdc  Wednesday   107215.0           West_CDMX_Pricing   

      hour   KM      mean                     ...                             \
                       BF base_fare min_fare  ...           ASP          vbf   
0        5    1  0.104919   2150.00   6150.0  ...   6975.000000   660.000000   
1        5   10  0.113024   2150.00   6150.0  ...  15065.000000  1530.000000   
2        5   16  0.101462   2150.00   6150.0  ...  12810.000000  1180.000000   
3        5    2  0.083296   2150.00   6150.0  ...  11140.000000   736.666667   
4        5    3  0.113976   2150.00   6150.0  ...   8210.000000   840.000000   
...    ...  ...       ...       ...      ...  ...           ...          ...   
39865   23    6  0.102876      6.87     29.9  ...     96.954286     7.617143   
39866   23    7  0.119826      6.87     29.9  ...     98.733333     9.031667   
39867   23    8  0.124295      6.87     29.9  ...    100.342500    10.250833   
39868   23    9  0.143700      6.87     29.9  ...    112.460000    14.130000   
39869   23  >30  0.179717      6.87     29.9  ...    247.080000    37.640000   

      count                                                     date  
         BF base_fare min_fare per_km per_minute ASP vbf              
0         2         2        2      2          2   2   2  2023-04-17  
1         2         2        2      2          2   2   2  2023-04-17  
2         1         1        1      1          1   1   1  2023-04-17  
3         3         3        3      3          3   3   3  2023-04-17  
4         1         1        1      1          1   1   1  2023-04-17  
...     ...       ...      ...    ...        ...  ..  ..         ...  
39865     7         7        7      7          7   7   7  2023-04-17  
39866     6         6        6      6          6   6   6  2023-04-17  
39867    12        12       12     12         12  12  12  2023-04-17  
39868     1         1        1      1          1   1   1  2023-04-17  
39869     1         1        1      1          1   1   1  2023-04-17  

[39870 rows x 22 columns]

In [42]:
# Save fares to new csv file df_weekdays

filename = "fares_geo_km_day_hour" + date_name + ".csv"

#df_weekdays.to_csv("outputs\\fares\\fares_weekday\\" + filename, encoding="utf8", index = False)
#df_geo.to_csv("../outputs/fares/fares_geo/" + filename, encoding="utf8", index = False)


#Save to Weekly data folder
df_geo_full.to_csv(newpath + filename, encoding="utf8", index = False)


# Append fares to existing csv file with historical fares
#filename = "fares_geo_hist.csv"

#df_weekdays.to_csv("outputs\\fares\\fares_weekday\\" + filename,                     # Add new data vertically
# df_geo.to_csv("../outputs/fares/fares_geo/" + filename,                     # Add new data vertically
#                     mode = 'a',
#                     header = False,
#                     encoding="utf8", 
#                     index = False)